# Library

In [ ]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm einops joblib icecream  -qq -U

     |████████████████████████████████| 1.7 MB 13.9 MB/s 
     |████████████████████████████████| 21.8 MB 1.2 MB/s 
     |████████████████████████████████| 102 kB 70.6 MB/s 
     |████████████████████████████████| 431 kB 78.0 MB/s 
     |████████████████████████████████| 97 kB 9.9 MB/s 
     |████████████████████████████████| 143 kB 69.0 MB/s 
     |████████████████████████████████| 180 kB 72.0 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 


In [ ]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from sklearn.metrics import f1_score

# ENV

In [ ]:

# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
# FP16 = True
FP16 = False


CONFIG = dict(
    seed=42,
    nickname = 'SwinB finecutmix',
    note = 'finecutmix',
    backbone='swin_base_patch4_window12_384',
    embedder= None,
    train_batch_size=8,
    valid_batch_size=16,
    img_size=384,
    num_epochs=50,
    early_stopping = False,
    early_stopping_step = 5,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes = 25,
    weight_decay=1e-6,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition='lg',
    _wandb_kernel='deb'
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# SET SEED 

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(CONFIG['seed'])


# Read the Data


In [24]:
import wandb
run = wandb.init(project="lg",
                 entity="jiwon7258",
                 config=CONFIG,
                 job_type='train',
                #  id='2ow4king',
                #  resume='must',
                 )
dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg_train:v0', type='dataset')

run.name = CONFIG['nickname']

# Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


Train Accuracy,0.83901
Train F1,0.80408
Train Loss,0.47385
Valid Accuracy,1
Valid F1,1
Valid Loss,0.0009


wandb: Downloading large artifact lg_train:v0, 918.24MB. 17301 files... Done. 0:0:0


In [6]:
TRAIN_PATH = dataset_dir
# TEST_PATH = dataset_dir / 'test'

# Augmentations

In [7]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15,
                           rotate_limit=35, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15,
                   b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()], p=1.),

    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(),
        ToTensorV2()], p=1.)
}


# Dataset

In [8]:
train_csv = sorted(glob(str(TRAIN_PATH / '*/*.csv')))
train_jpg = sorted(glob(str(TRAIN_PATH / '*/*.jpg')))
train_json = sorted(glob(str(TRAIN_PATH / '*/*.json')))


crops = []
diseases = []
risks = []
labels = []

for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
    
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
        
label_unique = sorted(np.unique(labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in labels] # len = train_len

In [ ]:
import joblib
joblib.dump(label_unique, 'label_unique')
wandb.save('label_unique')

In [10]:
train_jpg = np.array(train_jpg)
train_labels = np.array(train_labels)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, train_img, train_label, transforms=None):
        self.imgs = train_img
        self.labels = train_label
        self.transforms = transforms
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index):
        img_path = self.imgs[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, target
    
# trainDataset = CustomDataset(X_train, y_train, transforms = data_transforms['train'])
# trainDataloader = DataLoader(
#     trainDataset, batch_size=CONFIG['train_batch_size'], shuffle=True)

# validDataset = CustomDataset(X_val, y_val, transforms = data_transforms['valid'])
# validDataloader = DataLoader(validDataset, batch_size = CONFIG['valid_batch_size'], shuffle = True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
train_datasets = []
valid_datasets = []
train_dataloaders = []
valid_dataloaders = []

skf = StratifiedKFold(n_splits = 5)

for step, (train_index, val_index) in enumerate(skf.split(X = train_jpg, y= train_labels)):
    X_train = train_jpg[train_index]
    y_train = train_labels[train_index]
    X_val = train_jpg[val_index]
    y_val = train_labels[val_index]
    train_datasets.append(CustomDataset(
        X_train, y_train, transforms=data_transforms['train']))
    valid_datasets.append(CustomDataset(
        X_val, y_val, transforms=data_transforms['valid']))
    train_dataloaders.append(DataLoader(
        train_datasets[step], batch_size=CONFIG['train_batch_size'], shuffle=True)
    )
    valid_dataloaders.append(
        DataLoader(
            valid_datasets[step], batch_size=CONFIG['valid_batch_size'], shuffle=True)
    )


# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.n_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        output = self.fc(features)
        return output


model = Model(CONFIG['backbone'], CONFIG['embedder'], pretrained = True)
model.to(CONFIG['device'])
;

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window12_384_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_base_patch4_window12_384_22kto1k.pth


''

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])


In [ ]:
def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss()(logits.view(-1,CONFIG['num_classes']), targets.view(-1))

# CutMix

In [26]:
def cutmix(img, target):
    """ 
    img : (bs, C, H, W)
    target
        - (bs,)
        - integer scalar
    """
    batch_size, C, H, W, = img.shape
    # ic(img.shape)

    img_a = img
    target_a = target
    img_b = img
    target_b = target

    mask = np.arange(batch_size)
    mask = np.random.permutation(mask)
    # ic(mask)
    img_b = img_a[mask]
    target_b = target_a[mask]
    # ic(target, target_b)

    lam = np.random.uniform(low=0.3, high=0.7)
    r_x = np.random.uniform(low=0, high=W)
    r_y = np.random.uniform(low=0, high=H)
    r_w = W * np.sqrt(1 - lam)
    r_h = H * np.sqrt(1 - lam)
    ic(lam, r_x, r_y, r_w, r_h)
    x1 = np.int(np.clip((r_x - r_w) / 2, 0, W))
    x2 = np.int(np.clip((r_x + r_w) / 2, 0, W))
    y1 = np.int(np.clip((r_y - r_h) / 2, 0, H))
    y2 = np.int(np.clip((r_y + r_h) / 2, 0, H))
    ic(x1, x2, y1, y2)

    img_a[:, :, y1:y2, x1:x2] = img_b[:, :, y1:y2, x1:x2]

    # Adjust lambda to exact ratio

    lam = 1 - (x2 - x1) * (y2 - y1) / float(W * H)

    return img_a, target_b, lam


   # Training Function

In [27]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    if(FP16):
        scaler = amp.GradScaler()

    losses = AverageMeter()
    accuracy = AverageMeter()
    f1 = AverageMeter()

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img, target_b, lam = cutmix(img, target)

        img = img.to(device)
        target = target.to(device)
        target_b = target_b.to(device)

        batch_size = img.shape[0]

        if(FP16):
            with amp.autocast(enabled=True):
                logits = model(img)
                loss = criterion(logits, target) * lam + \
                    criterion(logits, target_b) * (1-lam)

                # loss를 Scale
                # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
                scaler.scale(loss).backward()
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
                # otherwise, optimizer.step() is skipped.
                scaler.step(optimizer)

                # Updates the scale for next iteration.
                scaler.update()

        else:
            logits = model(img)
            loss = criterion(logits, target) * lam + \
                criterion(logits, target_b) * (1-lam)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        losses.update(loss.item())

        # logits
        logits = logits.detach().cpu()

        # acc, f1
        probs = torch.softmax(logits, dim = -1)
        output = np.argmax(probs, axis=-1)
        output_b = np.argsort(probs)[:,-2]
        if (lam >= 0.5):
            step_acc = np.mean(
                output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
            step_f1 = f1_score(output.view(-1).numpy(),
                               target.view(-1).detach().cpu().numpy(), average='macro')
            step_acc_b = np.mean(
                output_b.view(-1).numpy() == target_b.view(-1).detach().cpu().numpy())
            step_f1_b = f1_score(output_b.view(-1).numpy(),
                                 target_b.view(-1).detach().cpu().numpy(), average='macro')
        else:
            step_acc = np.mean(
                output.view(-1).numpy() == target_b.view(-1).detach().cpu().numpy())
            step_f1 = f1_score(output.view(-1).numpy(),
                               target_b.view(-1).detach().cpu().numpy(), average='macro')
            step_acc_b = np.mean(
                output_b.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
            step_f1_b = f1_score(output_b.view(-1).numpy(),
                                 target.view(-1).detach().cpu().numpy(), average='macro')

                                 
        step_acc = step_acc * lam + step_acc_b * (1-lam)
        step_f1 = step_f1 * lam + step_f1_b * (1-lam)

        accuracy.update(step_acc)
        f1.update(step_f1)

        # loss
        train_loss = losses.avg
        train_acc = accuracy.avg
        train_f1 = f1.avg

        bar.set_postfix(
            Epoch=epoch, Train_Loss=train_loss, LR=optimizer.param_groups[
                0]["lr"], accuracy=train_acc, f1=train_f1
        )

    # Garbage Collector
    gc.collect()

    return losses.avg, accuracy.avg, f1.avg


   # Validation Function

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    losses = AverageMeter()
    accuracy = AverageMeter()
    f1 = AverageMeter()

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img = img.to(device)
        target = target.to(device)

        batch_size = img.shape[0]

        logits = model(img)
        loss = criterion(logits, target)

        # loss.item()은 loss를 Python Float으로 반환
        losses.update(loss.item())

        # logits
        logits = logits.detach().cpu()

        output = np.argmax(torch.softmax(logits, dim=-1), axis=-1)
        step_acc = np.mean(
            output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
        step_f1 = f1_score(output.view(-1).numpy(),
                           target.view(-1).detach().cpu().numpy(), average='macro')

        accuracy.update(step_acc)
        f1.update(step_f1)

        # loss
        val_loss = losses.avg
        val_acc = accuracy.avg
        val_f1 = f1.avg

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[
                0]["lr"], accuracy=val_acc, f1=val_f1
        )

    gc.collect()

    return losses.avg, accuracy.avg, f1.avg


In [25]:

def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
    START_EPOCH = 0,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(START_EPOCH, START_EPOCH + num_epochs):
        gc.collect()

        fold_num = 5
        fold = epoch % fold_num

        # for fold in range(fold_num) :

        trainDataloader = train_dataloaders[fold]
        validDataloader = valid_dataloaders[fold]

        train_train_loss, train_accuracy, train_f1 = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=trainDataloader,
            device=device,
            epoch=epoch,
        )

        val_loss, val_accuracy, val_f1 = valid_one_epoch(
            model, validDataloader, device=device, epoch=epoch
        )


        history[f"{metric_prefix}Train Loss"].append(train_train_loss)
        history[f"{metric_prefix}Train Accuracy"].append(train_accuracy)
        history[f"{metric_prefix}Train F1"].append(train_f1)
        history[f"{metric_prefix}Valid Loss"].append(val_loss)
        history[f"{metric_prefix}Valid Accuracy"].append(val_accuracy)
        history[f"{metric_prefix}Valid F1"].append(val_f1)

        # Log the metrics
        wandb.log(
            {
                f"{metric_prefix}Train Loss": train_train_loss,
                f"{metric_prefix}Valid Loss": val_loss,
                f"{metric_prefix}Train Accuracy": train_accuracy,
                f"{metric_prefix}Valid Accuracy": val_accuracy,
                f"{metric_prefix}Train F1": train_f1,
                f"{metric_prefix}Valid F1": val_f1,
            }
        )

        print(f"Valid Loss : {val_loss}")

        torch.save(model.state_dict(), f'last.bin')
        wandb.save(f'last.bin')

        # deep copy the model
        if val_loss <= best_loss:
            early_stop_counter = 0

            print(
                f"Validation Loss improved( {best_loss} ---> {val_loss}  )"
            )

            # Update Best Loss
            best_loss = val_loss

            # Update Best Model Weight
            # run.summary['Best RMSE'] = best_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            PATH = "{}epoch{:.0f}_Loss{:.4f}.bin".format(
                file_prefix, epoch, best_loss)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            wandb.save(PATH)

            print(f"Model Saved")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

        START_EPOCH = epoch + 1
        # break

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history


 # Run training 20 Epochs

In [ ]:
wandb.restore('final.bin', 'jiwon7258/lg/1ycjalgj', root='./')
model.load_state_dict(torch.load('final.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=1e-6, weight_decay=CONFIG['weight_decay'])


In [ ]:
ic.disable()
run_training(
    model=model,
    optimizer=optimizer,
    # scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    scheduler = None,
    device=device,
    num_epochs=20,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
    START_EPOCH=0
)


In [ ]:
torch.save(model.state_dict(), 'final.bin')
wandb.save('final.bin')


['/content/wandb/run-20220126_074607-1ycjalgj/files/final.bin']

# 60 epoch (~80)

In [ ]:
model.load_state_dict(torch.load('last.bin',
                      map_location=CONFIG['device']))

<All keys matched successfully>

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=1e-6, weight_decay=CONFIG['weight_decay'])


In [ ]:
ic.disable()
run_training(
    model=model,
    optimizer=optimizer,
    # scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    scheduler = None,
    device=device,
    num_epochs=60,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
    START_EPOCH=20
)


[INFO] Using GPU:Tesla P100-PCIE-16GB



100%|██████████| 73/73 [00:41<00:00,  1.74it/s, Epoch=20, LR=1e-6, Valid_Loss=0.00395, accuracy=1, f1=1]


Valid Loss : 0.003950121209714307
Validation Loss improved( inf ---> 0.003950121209714307  )
Model Saved


100%|██████████| 73/73 [00:41<00:00,  1.74it/s, Epoch=21, LR=1e-6, Valid_Loss=0.00443, accuracy=1, f1=1]


Valid Loss : 0.004431140178508342


...

100%|██████████| 73/73 [00:42<00:00,  1.73it/s, Epoch=78, LR=1e-6, Valid_Loss=0.000796, accuracy=1, f1=1]


Valid Loss : 0.0007955803199390536
Validation Loss improved( 0.0008601167674610161 ---> 0.0007955803199390536  )
Model Saved


100%|██████████| 73/73 [00:42<00:00,  1.73it/s, Epoch=79, LR=1e-6, Valid_Loss=0.000898, accuracy=1, f1=1]


Valid Loss : 0.000898455188702154
Training complete in 8h 51m 26s
Best Loss: 0.0008


(Model(
   (backbone): SwinTransformer(
     (patch_embed): PatchEmbed(
       (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
       (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
     )
     (pos_drop): Dropout(p=0.0, inplace=False)
     (layers): Sequential(
       (0): BasicLayer(
         dim=128, input_resolution=(96, 96), depth=2
         (blocks): ModuleList(
           (0): SwinTransformerBlock(
             (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
             (attn): WindowAttention(
               (qkv): Linear(in_features=128, out_features=384, bias=True)
               (attn_drop): Dropout(p=0.0, inplace=False)
               (proj): Linear(in_features=128, out_features=128, bias=True)
               (proj_drop): Dropout(p=0.0, inplace=False)
               (softmax): Softmax(dim=-1)
             )
             (drop_path): Identity()
             (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
             (ml

# 60 epoch (~ 140)

In [20]:
wandb.restore('last.bin', 'jiwon7258/lg/1ycjalgj', root='./')

<_io.TextIOWrapper name='./last.bin' mode='r' encoding='UTF-8'>

In [21]:
model.load_state_dict(torch.load('last.bin',
                      map_location=CONFIG['device']))

<All keys matched successfully>

In [22]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=1e-6, weight_decay=CONFIG['weight_decay'])


In [ ]:
ic.disable()
run_training(
    model=model,
    optimizer=optimizer,
    # scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    scheduler = None,
    device=device,
    num_epochs=100,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
    START_EPOCH=80
)


[INFO] Using GPU:Tesla T4



100%|██████████| 73/73 [01:04<00:00,  1.13it/s, Epoch=80, LR=1e-6, Valid_Loss=0.000657, accuracy=1, f1=1]


Valid Loss : 0.00065714538165955
Validation Loss improved( inf ---> 0.00065714538165955  )
Model Saved


  3%|▎         | 19/577 [00:24<11:49,  1.27s/it, Epoch=81, LR=1e-6, Train_Loss=0.567, accuracy=0.853, f1=0.805]